In [1]:
with open("Data/Day_22_content.txt") as file:
    initial_secrets = list(map(int, [line.strip() for line in file.readlines()]))

## Part 1

In [2]:
results = []
MOD = 16777216  # 2^24
for secret in initial_secrets:
    for _ in range(2000):
        
        # Step 1: Multiply by 64, XOR, prune
        secret = (secret ^ (secret * 64)) % MOD
            
        # Step 2: Divide by 32 (floor), XOR, prune
        secret = (secret ^ (secret // 32)) % MOD
            
        # Step 3: Multiply by 2048, XOR, prune
        secret = (secret ^ (secret * 2048)) % MOD
        
    results.append(secret)

In [3]:
sum(results)

14691757043

## Part 2

In [4]:
import numpy as np
from itertools import product

In [5]:
def generate_prices_vectorized(secret_numbers, steps=2000):
    secrets = np.array(secret_numbers, dtype = np.uint64)
    prices = np.zeros((len(secret_numbers), steps + 1), dtype = np.uint8)
    MOD = 16777216
    
    for step in range(steps + 1):
        # Generate price
        
        # Step 1: Multiply by 64, XOR, prune
        secrets = (secrets ^ (secrets * 64)) % MOD
            
        # Step 2: Divide by 32 (floor), XOR, prune
        secrets = (secrets ^ (secrets // 32)) % MOD
            
        # Step 3: Multiply by 2048, XOR, prune
        secrets = (secrets ^ (secrets * 2048)) % MOD
        prices[:, step] = secrets % 10

    return prices

In [6]:
# Generate prices and price changes
prices = generate_prices_vectorized(initial_secrets)
changes = np.diff(prices, axis = 1)

# All possible 4-change sequences (-9 to 9)
all_sequences = list(product(range(-9, 10), repeat=4))
all_sequences_array = np.array(all_sequences, dtype=np.int8)
sequence_length = all_sequences_array.shape[1]

# Initialize max banana total and optimal sequence
max_bananas = 0
best_sequences = None

In [10]:
for seq_idx in range(all_sequences_array.shape[0]):
    sequence_array = all_sequences_array[seq_idx]
    total_bananas = 0
    
    # Use broadcasting and vectorization
    for buyer_idx in range(changes.shape[0]):
        buyer_changes = changes[buyer_idx]
        
        # Use sliding window view to compare sequences efficiently
        window_view = np.lib.stride_tricks.sliding_window_view(buyer_changes, sequence_length)
        
        # Use broadcasting to find matching sequences
        matches = np.all(window_view == sequence_array, axis=1)
        
        # Find first occurence and update total bananas if match is found
        match_indices = np.where(matches)[0]
        if match_indices.size > 0:
            first_match_idx = match_indices[0]
            total_bananas += prices[buyer_idx, first_match_idx + sequence_length]
            
    if total_bananas > max_bananas:
        max_bananas = total_bananas
        best_sequences = sequence_array.tolist()
        

KeyboardInterrupt: 

In [ ]:
max_bananas